In [3]:
# inserting random numbers of objects, in random places in one image
import json
import random
import os
import math
from PIL import Image

def organize_points_clockwise(points):
    centroid = [sum(x[0] for x in points) / len(points), sum(x[1] for x in points) / len(points)]
    sorted_points = sorted(points, key=lambda p: math.atan2(p[1] - centroid[1], p[0] - centroid[0]))
    return sorted_points

def create_polygon_points(x, y, width, height):
    return organize_points_clockwise([
        [x, y],
        [x + width, y],
        [x + width, y + height],
        [x, y + height]
    ])

# Load the annotation file
with open('0A0WG84517.json', 'r') as file:
    annotation_data = json.load(file)

# Load the background image
background_image_path = annotation_data["imagePath"]
background_image = Image.open(background_image_path)

# Get Tote dimensions from the annotation
tote_points = annotation_data["shapes"][0]["points"]
tote_x_min = min(point[0] for point in tote_points)
tote_x_max = max(point[0] for point in tote_points)
tote_y_min = min(point[1] for point in tote_points)
tote_y_max = max(point[1] for point in tote_points)
tote_width = tote_x_max - tote_x_min
tote_height = tote_y_max - tote_y_min

# Set mean and standard deviation for the normal distribution
mean_objects = 5
std_dev_objects = 2
# Set an offset to move objects away from the edge
offset = 50

# Specify the path to your object images directory
object_images_directory = '/home/asma/Desktop/github/object insertion/out'

# List all files in the object images directory
object_images = [file for file in os.listdir(object_images_directory) if file.endswith('.jpg')]

# Generate a random number of objects based on a normal distribution
num_objects = round(random.gauss(mean_objects, std_dev_objects))

# Loop through each randomly selected object
for _ in range(num_objects):
    # Randomly pick an object from the list
    object_name = random.choice(object_images)

    # Load object image
    object_path = os.path.join(object_images_directory, object_name)
    object_image = Image.open(object_path)

    # Get random coordinates within the Tote with an offset
    x_position = random.randint(tote_x_min + offset, tote_x_max - offset - object_image.width)
    y_position = random.randint(tote_y_min + offset, tote_y_max - offset - object_image.height)

    # Paste the object on the background
    background_image.paste(object_image, (x_position, y_position))

    # Modify the annotation data for the new object
    new_object_annotation = {
        "label": "Object",
        "points": create_polygon_points(x_position, y_position, object_image.width, object_image.height),
        "group_id": None,
        "shape_type": "polygon",
        "flags": {}
    }

    # Append the new object annotation to the existing annotation data
    annotation_data["shapes"].append(new_object_annotation)

# Save the modified background image
background_image.save('output_image.jpg')

# Save the modified annotation data
with open('output_annotation.json', 'w') as output_file:
    json.dump(annotation_data, output_file, indent=2)


In [11]:
#loop over image folder
import json
import random
import os
import math
from PIL import Image

def organize_points_clockwise(points):
    centroid = [sum(x[0] for x in points) / len(points), sum(x[1] for x in points) / len(points)]
    sorted_points = sorted(points, key=lambda p: math.atan2(p[1] - centroid[1], p[0] - centroid[0]))
    return sorted_points

def create_polygon_points(x, y, width, height):
    return organize_points_clockwise([
        [x, y],
        [x + width, y],
        [x + width, y + height],
        [x, y + height]
    ])

def insert_objects(background_image_path, annotation_path, output_folder, object_images_directory):
    try:
        # Load the annotation file
        with open(annotation_path, 'r') as file:
            annotation_data = json.load(file)

        # Load the background image
        background_image = Image.open(background_image_path)

        # Get Tote dimensions from the annotation
        tote_points = annotation_data["shapes"][0]["points"]
        tote_x_min = min(point[0] for point in tote_points)
        tote_x_max = max(point[0] for point in tote_points)
        tote_y_min = min(point[1] for point in tote_points)
        tote_y_max = max(point[1] for point in tote_points)
        tote_width = tote_x_max - tote_x_min
        tote_height = tote_y_max - tote_y_min

        # Set mean and standard deviation for the normal distribution
        mean_objects = 5
        std_dev_objects = 2
        # Set an offset to move objects away from the edge
        offset = 50

        # Generate a random number of objects based on a normal distribution
        num_objects = round(random.gauss(mean_objects, std_dev_objects))

        print(f"Processing {background_image_path} with {num_objects} objects.")

        # Loop through each randomly selected object
        for _ in range(num_objects):
            # Randomly pick an object from the list
            object_name = random.choice(os.listdir(object_images_directory))

            # Load object image
            object_path = os.path.join(object_images_directory, object_name)
            object_image = Image.open(object_path)

            # Get random coordinates within the Tote with an offset
            x_position = random.randint(tote_x_min + offset, tote_x_max - offset - object_image.width)
            y_position = random.randint(tote_y_min + offset, tote_y_max - offset - object_image.height)

            print(f"Inserting object {object_name} at position ({x_position}, {y_position}).")

            # Paste the object on the background
            background_image.paste(object_image, (x_position, y_position))

            # Modify the annotation data for the new object
            new_object_annotation = {
                "label": "Object",
                "points": create_polygon_points(x_position, y_position, object_image.width, object_image.height),
                "group_id": None,
                "shape_type": "polygon",
                "flags": {}
            }

            # Append the new object annotation to the existing annotation data
            annotation_data["shapes"].append(new_object_annotation)

        # Save the modified background image
        output_image_path = os.path.join(output_folder, os.path.basename(background_image_path))
        background_image.save(output_image_path)

        # Save the modified annotation data
        output_annotation_path = os.path.join(output_folder, os.path.basename(annotation_path))
        with open(output_annotation_path, 'w') as output_file:
            json.dump(annotation_data, output_file, indent=2)

    except Exception as e:
        print(f"Error processing {background_image_path}: {str(e)}")

# Path to the folder containing background images
background_images_folder = '/home/asma/Desktop/github/object insertion/train_images'
# Path to the folder containing annotation files
annotations_folder = '/home/asma/Desktop/github/object insertion/train_annotations'

# Output folder for the modified images and annotations
output_folder = '/home/asma/Desktop/github/object insertion/newImg'

# Path to the object images directory
object_images_directory = '/home/asma/Desktop/github/object insertion/out'

# Loop through each background image and its corresponding annotation file
for background_image_name in os.listdir(background_images_folder):
    if background_image_name.endswith('.jpg'):
        background_image_path = os.path.join(background_images_folder, background_image_name)
        annotation_name = os.path.splitext(background_image_name)[0] + '.json'
        annotation_path = os.path.join(annotations_folder, annotation_name)

        # Check if the corresponding annotation file exists
        if os.path.exists(annotation_path):
            # Perform object insertion and save the modified image and annotation
            insert_objects(background_image_path, annotation_path, output_folder, object_images_directory)


Processing /home/asma/Desktop/github/object insertion/train_images/IT58YQAZYV.jpg with 5 objects.
Inserting object I8XRO69S35_Object_obj1_f20a8494.jpg at position (1394, 827).
Inserting object 3Z27746WUC_Object_obj2_8b10ffc2.jpg at position (1193, 807).
Inserting object LPSTFXNZ2U_Object_obj3_019949fb.jpg at position (1129, 1054).
Inserting object N1X665MNR1_Object_obj10_892d292b.jpg at position (480, 581).
Inserting object 7FZ53HWSG4_Object_obj3_fe3af49a.jpg at position (1794, 980).
Processing /home/asma/Desktop/github/object insertion/train_images/VR5V1Z6856.jpg with 7 objects.
Inserting object LJBEM18B3V_Object_obj2_a29c568d.jpg at position (1226, 675).
Inserting object ZGZ4Y7VIW8_Object_obj8_535c9dc0.jpg at position (791, 487).
Inserting object Q1EG9JU7HQ_Object_obj3_190456ef.jpg at position (1243, 593).
Inserting object WC65LUGQU1_Object_obj3_d1fb578a.jpg at position (1986, 1126).
Inserting object IRJS8TLXJJ_Object_obj8_bbaf335a.jpg at position (570, 1331).
Inserting object KYHWR6